# loader

> Find the relevant Jinja template location and load that into the environment.

In [ ]:
#| default_exp loader

In [ ]:
#| export
import os
from fastcore.all import Path, test_eq
from contextlib import contextmanager
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader, ChoiceLoader

In [ ]:
#| export
@contextmanager
def _set_directory(path: Path):
    """Changes the directory within the context"""
    origin = Path().absolute()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(origin)

In [ ]:
#| export
def _return_pth(pth:Path):
    "return the path if it exists.  If yaml file, allow suffix to be .yml or .yaml"
    pth = Path(pth)
    suffix = pth.suffix
    alt_suffix = '.yaml' if suffix == '.yml' else '.yml'
    alt_pth = pth.parent/f'{pth.stem}{alt_suffix}'
    
    if pth.exists(): 
        return pth
    elif alt_pth.exists(): 
        return alt_pth

In [ ]:
#|hide

# doesn't matter if suffix is .yml or .yaml
test_eq(_return_pth('_test_proj/_quarto.yaml'), _return_pth('_test_proj/_quarto.yml'))

In [ ]:
#|export
def _find_quarto_cfg(tpl_dir='_templates'):
    "find location of the `_quarto.yaml` file from the current directory."
    # Iterate through parent directories
    current_dir = Path(os.getcwd())
    while True:
        nm = _return_pth(current_dir/'_quarto.yml')
        if nm: 
            return nm
            
        # Move to the parent directory
        parent_dir = current_dir.parent

        # Check if the current directory is the root directory
        if parent_dir == current_dir: 
            break

        current_dir = parent_dir

In [ ]:
#|hide
with _set_directory('_test_proj/test/'):
    cfg_dir =  _find_quarto_cfg() 

# the _quarto.yml file lives in one directory up from `_test_proj/test/`
test_eq(cfg_dir.relative_to(os.getcwd()), Path('_test_proj/_quarto.yml'))

In [ ]:
#|export
def _proj_templates():
    "Returns path to local project quartodoc_templates/ directory if exists."
    # TODO: allow changing the location of templates folder via a config
    cfg = _find_quarto_cfg()
    if cfg: return cfg.parent/'quartodoc_templates/'

In [ ]:
#|hide
with _set_directory('_test_proj/test/'):
    pth = _proj_templates()
    
test_eq(pth.relative_to(os.getcwd()), Path('_test_proj/quartodoc_templates'))

# Render a Jinja Template

First, we must construct a Jinja environment by loading Jinja templates from the right location.  The following are the locations where templates are loaded from in order of precedence:
    
1. A `quartodoc_templates/` folder located at the root of your quarto project (in the same directory as `_quarto.yml`).
2. A `.quartodoc/templates` folder located in your home directory
3. The templates located in the `griffe_quarto` python package

In [ ]:
#|exporti
def env():
    "Constructs the Jinja environment with the right template loaders depending on the user's environment."
    base_loaders = [FileSystemLoader("~/.quartodoc/templates"), PackageLoader("griffe_quarto")]
    proj_tpl = _proj_templates()
    loaders = [FileSystemLoader(proj_tpl)] + base_loaders if proj_tpl else base_loaders
    
    return Environment(
        loader=ChoiceLoader(loaders),
        autoescape=select_autoescape(),
        trim_blocks=True,
        lstrip_blocks=True)

Consider the following example where we have two templates:

1. the parent template, `qmd.tpl` located in the `griffe_quarto` python package (in the `templates/` folder).
2. the child template, `test.tpl` located in this quarto project in the `quartodoc_templates/` folder.

In [ ]:
!cat ../griffe_quarto/templates/qmd.tpl

{% block frontmatter %}
---
title: {{ title }}
{% if description %}
description: {{ desc }}
{% endif %}
---
{% endblock frontmatter %}

{% block body %}{% endblock body %}


In [ ]:
!cat quartodoc_templates/test.tpl

{% extends "qmd.tpl" %}
{% block body %}
A test template with the variable `foo`: {{ foo }} 
Another line with the variable `bar`: {{ bar }}
{% endblock %}

**We can render these templates like so:**

Note that `test.tpl` inherits from `qmd.tpl`.

In [ ]:
template = env().get_template("test.tpl")

_rendered = template.render(title='A Test Title', 
                            description=None, 
                            foo='abc123', 
                            bar='xyz987')
print(_rendered)

---
title: A Test Title
---

A test template with the variable `foo`: abc123 
Another line with the variable `bar`: xyz987



In [ ]:
#|hide

test_eq(_rendered,
"""---
title: A Test Title
---

A test template with the variable `foo`: abc123 
Another line with the variable `bar`: xyz987
""")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()